In [11]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import itertools

In [ ]:
#|                |    predicted class    |
#|                |  1  |  2  |  3  |   4 |
#|                | hap | ang | sad | rel |
#|----------------|-----|-----|-----|-----|
#|          1 hap |  5  | -5  | -2  |  4  |
#| true     2 ang | -5  | 10  |  2  | -5  |
#| emotion  3 sad | -5  |  0  |  5  |  2  |
#| class    4 rel |  4  | -5  |  0  |  5  |

In [64]:
def get_reward(predictions, targets):
    reward = 0
    for pred, target in zip(predictions, targets):
        if target == 1:
            if pred == 1:
                reward += 5
            elif pred == 2:
                reward -= 5
            elif pred == 3:
                reward -= 2
            elif pred == 4:
                reward += 4
        elif target == 2:
            if pred == 1:
                reward -= 5
            elif pred == 2:
                reward += 10
            elif pred == 3:
                reward += 2
            elif pred == 4:
                reward -= 5
        elif target == 3:
            if pred == 1:
                reward -= 5
            elif pred == 3:
                reward += 5
            elif pred == 4:
                reward += 2
        elif target == 4:
            if pred == 1:
                reward += 4
            elif pred == 2:
                reward -= 5
            elif pred == 4:
                reward += 5
    return reward

In [12]:
train = pickle.load(open('phase_3_TRAIN_7d499bff69ca69b6_6372c3e_MLPC2021_pandas.pkl', 'rb'))
test = pickle.load(open('phase_4_CHALLENGE_f042c3290b44b1ef_e2a955e_MLPC2021_pandas.pkl', 'rb'))

In [40]:
test

,id,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,...,librosa_spectral_bandwidth_stdev,librosa_spectral_flatness_mean,librosa_spectral_flatness_stdev,midlevel_features_melody,midlevel_features_articulation,midlevel_features_rhythm_complexity,midlevel_features_rhythm_stability,midlevel_features_dissonance,midlevel_features_tonal_stability,midlevel_features_minorness
0,flagellates,0.095044,0.059303,4.363222,16.255108,3.3,0.483711,0.105291,791.174255,178.507248,...,377.285879,0.000013,0.000015,0.274462,-0.171781,-0.279024,-0.041689,-0.439434,0.348059,-0.124695
1,adjudicates,0.125889,0.057642,3.058106,91.297142,3.4,0.469212,0.123577,768.057068,183.266342,...,139.438996,0.000004,0.000004,0.234305,-0.117186,-0.198679,-0.015025,-0.399393,0.271292,0.000887
2,crowdfunded,0.122135,0.059374,4.632260,10.177483,1.7,0.522440,0.098636,1356.725464,426.218903,...,707.562829,0.000413,0.000438,0.222005,-0.322692,-0.355091,-0.094171,-0.361397,0.316675,-0.061305
3,easternmost,0.166465,0.066223,2.765269,171.956894,4.8,0.585742,0.111520,835.932190,188.463242,...,268.468795,0.000031,0.000038,0.041983,0.241189,-0.061846,0.018584,-0.234124,0.260259,-0.243865
4,dispatchers,0.144430,0.062011,3.318031,167.648773,2.8,0.566918,0.112361,688.516968,182.695404,...,119.744602,0.000004,0.000003,0.069976,-0.157599,-0.267475,-0.074170,-0.220617,0.290546,0.022956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,officialdom,0.083652,0.047943,5.752040,11.020640,1.3,0.540144,0.109967,724.627686,231.097778,...,676.448569,0.000032,0.000107,0.321545,-0.405632,-0.402905,-0.120522,-0.445618,0.337149,0.057841
1186,executrixes,0.148296,0.058452,2.567081,86.893875,3.9,0.502853,0.117577,779.646118,160.592804,...,124.770865,0.000004,0.000004,0.304416,-0.094806,-0.161916,-0.027787,-0.447481,0.323609,-0.152759
1187,hydrophobia,0.155180,0.072338,8.867934,67.109581,1.7,0.438148,0.144738,4556.834961,1674.448853,...,906.488450,0.016328,0.029121,-0.048243,-0.234154,-0.239152,-0.338395,-0.296404,0.177055,-0.189002
1188,cloudbursts,0.195466,0.059459,2.376977,260.545532,4.7,0.580569,0.104480,981.438171,126.935646,...,172.628509,0.000027,0.000023,0.058215,0.373557,0.030603,0.003979,-0.274888,0.296398,-0.245744


In [13]:
train

,id,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,...,midlevel_features_rhythm_complexity,midlevel_features_rhythm_stability,midlevel_features_dissonance,midlevel_features_tonal_stability,midlevel_features_minorness,score_mode,score_key_strength,mean_A,mean_V,quadrant
0,GG-01-000,0.206445,0.131383,13.998791,58.329521,3.1,0.539736,0.193987,3367.790527,3684.124268,...,-0.303987,-0.053306,-0.325228,0.347745,-0.119278,0.0,0.795727,50.689655,2.724138,1
1,GG-01-001,0.145753,0.057971,3.533264,75.166183,4.2,0.476832,0.123564,1395.528809,401.987610,...,-0.381182,0.120750,-0.479266,0.451572,-0.064252,0.0,0.844387,50.689655,2.724138,1
2,GG-01-002,0.149111,0.059011,3.337368,82.753929,4.3,0.457741,0.132359,1318.759644,289.306152,...,-0.337422,0.103853,-0.388214,0.430684,-0.226197,0.0,0.758119,50.689655,2.724138,1
3,GG-01-003,0.163914,0.057474,3.078172,79.024742,4.1,0.504427,0.121134,1258.258423,262.561310,...,-0.314282,0.118872,-0.335902,0.314371,-0.006619,1.0,0.747180,50.689655,2.724138,1
4,GG-01-004,0.157382,0.054116,2.682208,59.633064,3.9,0.503377,0.121658,1244.375122,260.756195,...,-0.302913,-0.002940,-0.395495,0.368037,-0.162711,0.0,0.710480,50.689655,2.724138,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3750,RT-47-029,0.106072,0.048556,4.600393,71.847145,1.7,0.541098,0.146353,3761.501709,1075.834717,...,-0.294453,-0.048271,-0.518002,0.364257,-0.344354,0.0,0.741726,20.344828,-0.896552,3
3751,RT-47-030,0.097130,0.045978,4.032829,81.727577,2.4,0.506956,0.129266,3283.780029,956.233765,...,-0.211686,-0.043546,-0.427122,0.318351,-0.382207,0.0,0.729813,20.344828,-0.896552,3
3752,RT-47-031,0.142735,0.059378,4.094213,27.877666,0.9,0.553664,0.155784,5419.066406,1376.261963,...,-0.268202,-0.223166,-0.392003,0.249261,-0.183019,0.0,0.752923,20.344828,-0.896552,3
3753,RT-47-032,0.134667,0.057092,5.675084,57.549480,1.4,0.524951,0.150398,4481.284668,1550.097534,...,-0.286899,-0.110805,-0.460086,0.320651,-0.221194,0.0,0.744074,20.344828,-0.896552,3


In [14]:
target_train_all = train.drop(columns=train.columns[:-1], axis=1)
train_id_all = train.drop(columns=train.columns[1:], axis=1)
train_all = train.drop(columns=train.columns[-5:], axis=1)
train_all = train_all.drop(columns=['id'], axis=1)

In [139]:
#training accuracy, whole train set used for training and testing
clf = XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=6)
clf.fit(train_all, target_train_all)
pred = clf.predict(train_all)
t = target_train_all.to_numpy().reshape(1, len(pred))[0]
acc = sum(pred == t) / len(pred)
print(acc)

[18:52:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1.0


# Artist Split 
### Im Dataset 6 Künstler, 5 werden fürs Training, 1 als Testset verwendet

In [46]:
artists = ['GG', 'FG', 'AH', 'SR', 'AS', 'RT']
counts = [0, 0, 0, 0, 0, 0]
for i, row in train_id_all.iterrows():
    index = artists.index(row.id[:2])
    counts[index] += 1

print(artists)
print(counts)

['GG', 'FG', 'AH', 'SR', 'AS', 'RT']
[370, 482, 426, 461, 382, 486]


In [78]:
def artist_split(artist_for_testset):
    one_artist = train.loc[train['id'].str.startswith(artist_for_testset)]
    the_other = train.loc[~train['id'].str.startswith(artist_for_testset)]

    target_one_artist = one_artist.drop(columns=one_artist.columns[:-1], axis=1)
    one_artist_id = one_artist.drop(columns=one_artist.columns[1:], axis=1)
    one_artist = one_artist.drop(columns=one_artist.columns[-5:], axis=1)
    one_artist = one_artist.drop(columns=['id'], axis=1)

    target_the_other = the_other.drop(columns=the_other.columns[:-1], axis=1)
    train_id_all = the_other.drop(columns=the_other.columns[1:], axis=1)
    the_other = the_other.drop(columns=the_other.columns[-5:], axis=1)
    the_other = the_other.drop(columns=['id'], axis=1)
    
    return the_other, target_the_other, one_artist, target_one_artist

In [103]:
#trained on 5 artist, tested on 1 artist
all_important_features = []

for i in ['GG', 'FG', 'AH', 'SR', 'AS', 'RT']:
    the_other, target_the_other, one_artist, target_one_artist = artist_split(i)
    clf = XGBClassifier(n_estimators=200, learning_rate=0.5, max_depth=12)
    clf.fit(the_other, target_the_other)
    pred = clf.predict(one_artist)
    
    feat_importance = clf.feature_importances_
    important_f = []
    for index, feat in enumerate(feat_importance):
        important_f.append([index, feat])
    important_f.sort(key=lambda x: x[1], reverse=True)
    all_important_features.append(important_f[:10])
    
    t = target_one_artist.to_numpy().reshape(1, len(pred))[0]
    acc = sum(pred == t) / len(pred)
    print(f'\n Artist {i} is the testset')
    print(acc)
    reward = get_reward(pred, t)
    print('Reward: ', reward)
    av_reward = reward/len(pred)
    print('Average Reward: ', av_reward)
    print('Derzeitiger average: ', 3500/1190)
    
print('\n Important features:')
all_important_features

[17:50:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Artist GG is the testset
0.5702702702702702
Reward:  1092
Average Reward:  2.9513513513513514
Derzeitiger average:  2.9411764705882355
[17:50:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Artist FG is the testset
0.7655601659751037
Reward:  1870
Average Reward:  3.879668049792531
Derzeitiger average:  2.9411764705882355
[17:50:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Sta

[[[4, 0.08574112],
  [81, 0.081809595],
  [78, 0.02684438],
  [9, 0.023416445],
  [10, 0.021972064],
  [82, 0.021920528],
  [17, 0.018608594],
  [22, 0.01626939],
  [15, 0.01521879],
  [62, 0.01491358]],
 [[4, 0.0849358],
  [81, 0.06235944],
  [78, 0.027900966],
  [82, 0.019303627],
  [19, 0.018608801],
  [58, 0.01809671],
  [7, 0.016808126],
  [27, 0.01648466],
  [76, 0.016269365],
  [57, 0.01579666]],
 [[81, 0.095900826],
  [4, 0.082098566],
  [82, 0.029998463],
  [76, 0.020497648],
  [7, 0.019598953],
  [5, 0.019081827],
  [9, 0.017968448],
  [10, 0.017811729],
  [17, 0.016191239],
  [3, 0.01507074]],
 [[81, 0.09281583],
  [4, 0.08151539],
  [82, 0.022692211],
  [65, 0.019517547],
  [76, 0.018859223],
  [15, 0.017922627],
  [7, 0.01756239],
  [84, 0.017292136],
  [22, 0.016281053],
  [17, 0.015466943]],
 [[4, 0.09693922],
  [81, 0.07717788],
  [82, 0.029889401],
  [7, 0.02334251],
  [76, 0.022026535],
  [62, 0.02177743],
  [10, 0.019199785],
  [5, 0.018457614],
  [17, 0.018211735],


# Melodie Split (einzelne Melodien gar nicht im Trainingsset, nur im Testset)

In [95]:
melodies = ['01', '02', '03', '04', '05', '08', '09', '11', '12', '14', '15', '16', '18', '19', '20', '21', '22', '24', '25', '26', '27', '28', '29', '30', '31', '32', '34', '35', '36', '37', '38', '39', '41','45', '46', '47']
melodie_counts = [0] * len(melodies)
for i, row in train_id_all.iterrows():
    index = melodies.index(row.id[3:5])
    melodie_counts[index] += 1

print(melodies)
print(melodie_counts)

['01', '02', '03', '04', '05', '08', '09', '11', '12', '14', '15', '16', '18', '19', '20', '21', '22', '24', '25', '26', '27', '28', '29', '30', '31', '32', '34', '35', '36', '37', '38', '39', '41', '45', '46', '47']
[70, 76, 54, 55, 37, 158, 35, 47, 79, 53, 142, 195, 32, 78, 35, 26, 39, 158, 53, 70, 29, 127, 22, 89, 65, 76, 82, 44, 103, 35, 76, 30, 36, 28, 79, 194]


In [98]:
my_melodie_test_sets = []
current_count = 0
current_set = []
for mel, cou in zip(melodies, melodie_counts):
    current_count += cou
    current_set.append(mel)
    if current_count > 500:
        current_count = 0
        my_melodie_test_sets.append(current_set)
        current_set = []
my_melodie_test_sets

[['01', '02', '03', '04', '05', '08', '09', '11'],
 ['12', '14', '15', '16', '18'],
 ['19', '20', '21', '22', '24', '25', '26', '27', '28'],
 ['29', '30', '31', '32', '34', '35', '36', '37']]

In [99]:
def melodie_split(melodie_set):
    melodie_test = train.loc[train['id'].str[3:5].isin(melodie_set)]
    melodie_train = train.loc[~train['id'].str[3:5].isin(melodie_set)]

    target_melodie_test = melodie_test.drop(columns=melodie_test.columns[:-1], axis=1)
    melodie_test_id = melodie_test.drop(columns=melodie_test.columns[1:], axis=1)
    melodie_test = melodie_test.drop(columns=one_artist.columns[-5:], axis=1)
    melodie_test = melodie_test.drop(columns=['id'], axis=1)

    target_melodie_train = melodie_train.drop(columns=melodie_train.columns[:-1], axis=1)
    melodie_train_id = melodie_train.drop(columns=melodie_train.columns[1:], axis=1)
    melodie_train = melodie_train.drop(columns=melodie_train.columns[-5:], axis=1)
    melodie_train = melodie_train.drop(columns=['id'], axis=1)
    
    return melodie_train, target_melodie_train, melodie_test, target_melodie_test

In [102]:
all_important_features = []
for i in my_melodie_test_sets:
    melodie_train, target_melodie_train, melodie_test, target_melodie_test = melodie_split(i)
    clf = XGBClassifier(n_estimators=500, learning_rate=0.5, max_depth=12)
    clf.fit(melodie_train, target_melodie_train)
    pred = clf.predict(melodie_test)

    feat_importance = clf.feature_importances_
    important_f = []
    for index, feat in enumerate(feat_importance):
        important_f.append([index, feat])
    important_f.sort(key=lambda x: x[1], reverse=True)
    all_important_features.append(important_f[:10])

    t = target_melodie_test.to_numpy().reshape(1, len(pred))[0]
    acc = sum(pred == t) / len(pred)
    print(f'\n Melodies {i} are the testset')
    print(acc)
    reward = get_reward(pred, t)
    print('Reward: ', reward)
    av_reward = reward/len(pred)
    print('Average Reward: ', av_reward)
    print('Derzeitiger average: ', 3500/1190)

print('\n Important features:')
print(all_important_features)

[17:48:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Melodies ['01', '02', '03', '04', '05', '08', '09', '11'] are the testset
0.5112781954887218
Reward:  1522
Average Reward:  2.8609022556390977
Derzeitiger average:  2.9411764705882355
[17:48:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Melodies ['12', '14', '15', '16', '18'] are the testset
0.718562874251497
Reward:  1575
Average Reward:  3.143712574850299
Derzeitiger average:  2.9411764705882355
[17:48:38] WARNING: C:/Use

[[[81, 0.109068036],
  [4, 0.09718218],
  [22, 0.02345755],
  [27, 0.020718316],
  [57, 0.020010006],
  [54, 0.017120818],
  [76, 0.016758362],
  [29, 0.016697524],
  [10, 0.015969034],
  [80, 0.01585223]],
 [[81, 0.08580675],
  [4, 0.074652076],
  [22, 0.020055652],
  [82, 0.019830564],
  [78, 0.019602044],
  [57, 0.017095633],
  [65, 0.016771518],
  [76, 0.016536256],
  [52, 0.01629433],
  [7, 0.016244287]],
 [[4, 0.08892325],
  [81, 0.08331343],
  [17, 0.031229129],
  [82, 0.029805835],
  [51, 0.02393392],
  [76, 0.020972406],
  [63, 0.02038981],
  [30, 0.017902251],
  [47, 0.017132934],
  [86, 0.016006941]],
 [[4, 0.082680635],
  [81, 0.079007134],
  [22, 0.024585392],
  [78, 0.021853343],
  [82, 0.02177354],
  [52, 0.021625658],
  [67, 0.02121357],
  [3, 0.019569078],
  [10, 0.019353135],
  [23, 0.019313635]]]

# Melodie Split mit zusätzlichen feature combinationen

In [137]:
def melodie_split_with_added_features(melodie_set):
    melodie_test = train.loc[train['id'].str[3:5].isin(melodie_set)]
    melodie_train = train.loc[~train['id'].str[3:5].isin(melodie_set)]

    target_melodie_test = melodie_test.drop(columns=melodie_test.columns[:-1], axis=1)
    melodie_test_id = melodie_test.drop(columns=melodie_test.columns[1:], axis=1)
    melodie_test = melodie_test.drop(columns=one_artist.columns[-5:], axis=1)
    melodie_test = melodie_test.drop(columns=['id'], axis=1)
    melodie_test_with_added_features = melodie_test.copy()
    combi = 1
    for i, j in list(itertools.permutations([4, 81, 82, 22], 2)):
        melodie_test_with_added_features[f'combination_{combi}'] = melodie_test_with_added_features.iloc[:,i] * melodie_test_with_added_features.iloc[:,j]
        combi += 1

    target_melodie_train = melodie_train.drop(columns=melodie_train.columns[:-1], axis=1)
    melodie_train_id = melodie_train.drop(columns=melodie_train.columns[1:], axis=1)
    melodie_train = melodie_train.drop(columns=melodie_train.columns[-5:], axis=1)
    melodie_train = melodie_train.drop(columns=['id'], axis=1)
    melodie_train_with_added_features = melodie_train.copy()
    combi = 1
    for i, j in list(itertools.permutations([4, 81, 82, 22], 2)):
        melodie_train_with_added_features[f'combination_{combi}'] = melodie_train_with_added_features.iloc[:,i] * melodie_train_with_added_features.iloc[:,j]
        combi += 1
    
    return melodie_train_with_added_features, target_melodie_train, melodie_test_with_added_features, target_melodie_test

In [138]:
all_important_features = []
for i in my_melodie_test_sets:
    melodie_train, target_melodie_train, melodie_test, target_melodie_test = melodie_split_with_added_features(i)
    clf = XGBClassifier(n_estimators=500, learning_rate=0.5, max_depth=12)
    clf.fit(melodie_train, target_melodie_train)
    pred = clf.predict(melodie_test)

    feat_importance = clf.feature_importances_
    important_f = []
    for index, feat in enumerate(feat_importance):
        important_f.append([index, feat])
    important_f.sort(key=lambda x: x[1], reverse=True)
    all_important_features.append(important_f[:10])

    t = target_melodie_test.to_numpy().reshape(1, len(pred))[0]
    acc = sum(pred == t) / len(pred)
    print(f'\n Melodies {i} are the testset')
    print(acc)
    reward = get_reward(pred, t)
    print('Reward: ', reward)
    av_reward = reward/len(pred)
    print('Average Reward: ', av_reward)
    print('Derzeitiger average: ', 3500/1190)

print('\n Important features:')
print(all_important_features)

c:\users\georg\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\georg\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[18:45:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Melodies ['01', '02', '03', '04', '05', '08', '09', '11'] are the testset
0.5112781954887218
Reward:  1465
Average Reward:  2.7537593984962405
Derzeitiger average:  2.9411764705882355
[18:45:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Melodies ['12', '14', '15', '16', '18'] are the testset
0.7564870259481038
Reward:  1615
Average Reward:  3.223552894211577
Derzeitiger average:  2.9411764705882355
[18:45:11] WARNING: C:/Us

# Für die Challenge

In [157]:
train = pickle.load(open('phase_3_TRAIN_7d499bff69ca69b6_6372c3e_MLPC2021_pandas.pkl', 'rb'))
test = pickle.load(open('phase_4_CHALLENGE_f042c3290b44b1ef_e2a955e_MLPC2021_pandas.pkl', 'rb'))

In [158]:
target_train_all = train.drop(columns=train.columns[:-1], axis=1)
train_id_all = train.drop(columns=train.columns[1:], axis=1)
train_all = train.drop(columns=train.columns[-5:], axis=1)
train_all = train_all.drop(columns=['id'], axis=1)
train_all

,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,essentia_spectral_complexity_mean,...,librosa_spectral_bandwidth_stdev,librosa_spectral_flatness_mean,librosa_spectral_flatness_stdev,midlevel_features_melody,midlevel_features_articulation,midlevel_features_rhythm_complexity,midlevel_features_rhythm_stability,midlevel_features_dissonance,midlevel_features_tonal_stability,midlevel_features_minorness
0,0.206445,0.131383,13.998791,58.329521,3.1,0.539736,0.193987,3367.790527,3684.124268,2.986095,...,1781.705789,0.139911,0.248388,0.065397,-0.029380,-0.303987,-0.053306,-0.325228,0.347745,-0.119278
1,0.145753,0.057971,3.533264,75.166183,4.2,0.476832,0.123564,1395.528809,401.987610,4.181923,...,721.292781,0.000275,0.000484,0.353040,-0.073957,-0.381182,0.120750,-0.479266,0.451572,-0.064252
2,0.149111,0.059011,3.337368,82.753929,4.3,0.457741,0.132359,1318.759644,289.306152,4.303592,...,583.569947,0.000202,0.000314,0.201885,0.016944,-0.337422,0.103853,-0.388214,0.430684,-0.226197
3,0.163914,0.057474,3.078172,79.024742,4.1,0.504427,0.121134,1258.258423,262.561310,5.271147,...,575.961419,0.000202,0.000257,0.181998,-0.019165,-0.314282,0.118872,-0.335902,0.314371,-0.006619
4,0.157382,0.054116,2.682208,59.633064,3.9,0.503377,0.121658,1244.375122,260.756195,4.122828,...,539.887336,0.000217,0.000241,0.256362,-0.151648,-0.302913,-0.002940,-0.395495,0.368037,-0.162711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3750,0.106072,0.048556,4.600393,71.847145,1.7,0.541098,0.146353,3761.501709,1075.834717,3.235226,...,885.326262,0.003383,0.003238,0.357967,-0.189399,-0.294453,-0.048271,-0.518002,0.364257,-0.344354
3751,0.097130,0.045978,4.032829,81.727577,2.4,0.506956,0.129266,3283.780029,956.233765,3.852839,...,890.114308,0.002536,0.002562,0.223821,-0.081319,-0.211686,-0.043546,-0.427122,0.318351,-0.382207
3752,0.142735,0.059378,4.094213,27.877666,0.9,0.553664,0.155784,5419.066406,1376.261963,2.230591,...,668.544678,0.010153,0.009679,0.190541,-0.334320,-0.268202,-0.223166,-0.392003,0.249261,-0.183019
3753,0.134667,0.057092,5.675084,57.549480,1.4,0.524951,0.150398,4481.284668,1550.097534,2.498262,...,1011.390115,0.006604,0.007357,0.292983,-0.208947,-0.286899,-0.110805,-0.460086,0.320651,-0.221194


In [159]:
test_id = test.drop(columns=test.columns[1:], axis=1)
test = test.drop(columns=test.columns[:1], axis=1)
test

,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,essentia_spectral_complexity_mean,...,librosa_spectral_bandwidth_stdev,librosa_spectral_flatness_mean,librosa_spectral_flatness_stdev,midlevel_features_melody,midlevel_features_articulation,midlevel_features_rhythm_complexity,midlevel_features_rhythm_stability,midlevel_features_dissonance,midlevel_features_tonal_stability,midlevel_features_minorness
0,0.095044,0.059303,4.363222,16.255108,3.3,0.483711,0.105291,791.174255,178.507248,1.526072,...,377.285879,0.000013,0.000015,0.274462,-0.171781,-0.279024,-0.041689,-0.439434,0.348059,-0.124695
1,0.125889,0.057642,3.058106,91.297142,3.4,0.469212,0.123577,768.057068,183.266342,3.728853,...,139.438996,0.000004,0.000004,0.234305,-0.117186,-0.198679,-0.015025,-0.399393,0.271292,0.000887
2,0.122135,0.059374,4.632260,10.177483,1.7,0.522440,0.098636,1356.725464,426.218903,0.922364,...,707.562829,0.000413,0.000438,0.222005,-0.322692,-0.355091,-0.094171,-0.361397,0.316675,-0.061305
3,0.166465,0.066223,2.765269,171.956894,4.8,0.585742,0.111520,835.932190,188.463242,6.529548,...,268.468795,0.000031,0.000038,0.041983,0.241189,-0.061846,0.018584,-0.234124,0.260259,-0.243865
4,0.144430,0.062011,3.318031,167.648773,2.8,0.566918,0.112361,688.516968,182.695404,5.961761,...,119.744602,0.000004,0.000003,0.069976,-0.157599,-0.267475,-0.074170,-0.220617,0.290546,0.022956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,0.083652,0.047943,5.752040,11.020640,1.3,0.540144,0.109967,724.627686,231.097778,1.033604,...,676.448569,0.000032,0.000107,0.321545,-0.405632,-0.402905,-0.120522,-0.445618,0.337149,0.057841
1186,0.148296,0.058452,2.567081,86.893875,3.9,0.502853,0.117577,779.646118,160.592804,4.100811,...,124.770865,0.000004,0.000004,0.304416,-0.094806,-0.161916,-0.027787,-0.447481,0.323609,-0.152759
1187,0.155180,0.072338,8.867934,67.109581,1.7,0.438148,0.144738,4556.834961,1674.448853,1.579374,...,906.488450,0.016328,0.029121,-0.048243,-0.234154,-0.239152,-0.338395,-0.296404,0.177055,-0.189002
1188,0.195466,0.059459,2.376977,260.545532,4.7,0.580569,0.104480,981.438171,126.935646,8.566628,...,172.628509,0.000027,0.000023,0.058215,0.373557,0.030603,0.003979,-0.274888,0.296398,-0.245744


In [160]:
clf = XGBClassifier(n_estimators=500, learning_rate=0.5, max_depth=6)
clf.fit(train_all, target_train_all)
pred = clf.predict(test)
pred

c:\users\georg\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\georg\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[19:38:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([3, 3, 3, ..., 3, 1, 4], dtype=int64)

In [161]:
test_id['quadrant'] = pred

In [163]:
test_id.to_csv(r'XGboost_results.csv', index = False)